# Targeted Maximum Likelihood Estimator
The targeted maximum likelihood estimator (TMLE) is a doubly robust estimator. What distinguishes it from other doubly robust estimators (augmented-IPTW) is that it uses a secondary targeting (hence the name) step that optimizes the bias-variance tradeoff for the target parameter. A common target parameter is the sample average treatment effect, which compares to counterfactual where all individuals in the study sample were treated versus the counterfactual where no individual was treated. Throughout this section, I will use the notation that is common in the TMLE literature. This is a little different from the notation in other documents.

The target parameter is defined as
$$\psi = E_L\left[E\left[Y^{a=1}|L\right] - E\left[Y^{a=0}|L\right]\right]$$
For continuous outcomes, this is the only parameter provided

This tutorial focuses on estimating TMLE with a continuous outcome. Behind the scenes, TMLE converts the outcome to be bounded between 0 and 1. This allows TMLE to be correctly estimated. This bounding will not impact the user results since `TMLE` bounds the outcome data and unbounds the results, $\psi$. Unlike binary outcomes, `TMLE` only provides the averge treatment effect, as defined above. I will demonstrate with both the default regression models and machine learning algorithms.

We will also build on the previous tutorial and account for missing at random data using IPMW

## Continuous Outcomes
To motivate our example, we will use a simulated data set included with *zEpid*. In the data set, we have a cohort of HIV-positive individuals. We are interested in the sample average treatment effect of antiretroviral therapy (ART) on CD4 T-cell count at 45 weeks. We will ignore competing risks for this example demonstration. Based on substantive background knowledge, we believe that the treated and untreated population are exchangeable based gender, age, CD4 T-cell count, and detectable viral load. 

If no custom model is specified, users can request either Normal or Poisson distributions for continuous outcomes. We will first demonstrate with an assumed normal distribution for the outcome model.

`TMLE` detects whether the outcome is continuous in the background, so you don't have to specify anything special for continuous outcome data

In [1]:
import zepid
from zepid import load_sample_data, spline
from zepid.causal.doublyrobust import TMLE

print(zepid.__version__)

0.9.0


In [2]:
df = load_sample_data(False)
df[['age_rs1', 'age_rs2']] = spline(df, 'age0', n_knots=3, term=2, restricted=True)
df[['cd4_rs1', 'cd4_rs2']] = spline(df, 'cd40', n_knots=3, term=2, restricted=True)

df = df.drop(columns=['dead'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 547 entries, 0 to 546
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        547 non-null    int64  
 1   male      547 non-null    int64  
 2   age0      547 non-null    int64  
 3   cd40      547 non-null    int64  
 4   dvl0      547 non-null    int64  
 5   art       547 non-null    int64  
 6   t         547 non-null    float64
 7   cd4_wk45  460 non-null    float64
 8   age_rs1   547 non-null    float64
 9   age_rs2   547 non-null    float64
 10  cd4_rs1   547 non-null    float64
 11  cd4_rs2   547 non-null    float64
dtypes: float64(6), int64(6)
memory usage: 55.6 KB


We are now ready to estimate TMLE with a continuous outcome. We will specify `missing_model()` to account for missing `cd4_wk45` data. We will use the default option, which assumes a Normal model for continuous outcomes

In [3]:
tml = TMLE(df, exposure='art', outcome='cd4_wk45')
tml.exposure_model('male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.missing_model('art + male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.outcome_model('art + male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.fit()
tml.summary()

                Targeted Maximum Likelihood Estimator                 
Treatment:        art             No. Observations:     547                 
Outcome:          cd4_wk45        No. Missing Outcome:  87                  
g-Model:          Logistic        Missing Model:        Logistic            
Q-Model:          gaussian       
Average Treatment Effect:  213.884
95.0% two-sided CI: (111.858 , 315.91)


The results indicate that ART increased CD4 T-cell count. 

## Poisson Distribution
We can also specify that the outcome data follows a Poisson distribution rather than Normally distributed outcomes

In [4]:
tml = TMLE(df, exposure='art', outcome='cd4_wk45')
tml.exposure_model('male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.missing_model('art + male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.outcome_model('art + male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', 
                  continuous_distribution='poisson', print_results=False)
tml.fit()
tml.summary()

                Targeted Maximum Likelihood Estimator                 
Treatment:        art             No. Observations:     547                 
Outcome:          cd4_wk45        No. Missing Outcome:  87                  
g-Model:          Logistic        Missing Model:        Logistic            
Q-Model:          poisson        
Average Treatment Effect:  213.877
95.0% two-sided CI: (111.88 , 315.874)


Overall, similar results are obtained between assuming Normal vs Poisson distributed outcomes.

# Conclusion
In this tutorial, I have described how TMLE can be estimated for continuous outcomes with *zEpid*. I demonstrated for parametric models with normal and Poisson distributed outcomes. Additionally, I demonstrated the usage of machine learning regressors with super learner.

## References

Schuler, Megan S., and Sherri Rose. "Targeted maximum likelihood estimation for causal inference in observational studies." American Journal of Epidemiology 185.1 (2017): 65-73.

van der Laan, Mark J., and Sherri Rose. Targeted learning: causal inference for observational and experimental data. Springer Science & Business Media, 2011.

van Der Laan, Mark J., and Daniel Rubin. "Targeted maximum likelihood learning." The International Journal of Biostatistics 2.1 (2006).

Gruber, S., & van der Laan, M. J. (2011). tmle: An R package for targeted maximum likelihood estimation.